In [4]:
# read csv for our data to analyze
import pandas as pd
import evaluate
import sklearn
experiment_df1 = pd.read_csv(f'experiment_sentembb_32_Roberta.csv')
preds = experiment_df1['predictions'].values.tolist()
labels = experiment_df1['labels'].values.tolist()


acc = sklearn.metrics.accuracy_score(labels, preds)
print(acc)

0.5311601150527325


In [83]:
# experiment dataframe
experiment_df1 

,Unnamed: 0,predictions,labels,selected_chunk_ids
0,0,2,1,"[5, 22, 33, 34, 59, 64, 86, 96, 98, 113, 157, ..."
1,1,2,1,"[195, 212, 217, 219, 245, 284, 289, 292, 300, ..."
2,2,0,2,"[375, 380, 392, 403, 404, 412, 414, 442, 469, ..."
3,3,3,3,"[565, 607, 678, 680, 682, 688, 709, 712, 717, ..."
4,4,0,0,"[799, 809, 822, 824, 826, 834, 836, 838, 848, ..."
...,...,...,...,...
2081,2081,0,3,"[325192, 325193, 325202, 325208, 325209, 32521..."
2082,2082,3,3,"[325280, 325281, 325282, 325289, 325296, 32529..."
2083,2083,1,1,"[325369, 325374, 325377, 325379, 325384, 32538..."
2084,2084,1,1,"[325458, 325467, 325472, 325474, 325478, 32550..."


In [9]:
# question data
import json
import pandas as pd
questionlist = []
# Open the JSON file
with open('datasets/quality/raw/QuALITY.v1.0.1.htmlstripped.dev', 'r') as file:
    # Load the JSON data
    for line in file:
      line = line.strip()

      json_obj = json.loads(line)

      article = json_obj['article']
      
      for jquestion in json_obj['questions']:
        # getting the relevant properties
        row = {'article': article, 'question': jquestion['question'], 'options': jquestion['options'], 'gold_label': jquestion['gold_label']}
        questionlist.append(row)

# turn it into hugging face dataset
question_df = pd.DataFrame(questionlist)

In [107]:
# question dataframe
question_df.loc[:5]

,article,question,options,gold_label
0,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,How much time has passed between Blake's night...,"[7 years, 10 hours, 12 years, 1 hour]",2
1,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Why does Deirdre get so upset when Blake Past ...,[Because Blake is trying to guilt Deirdre into...,2
2,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Why does shame flame in Blake's cheeks when De...,[He is embarrassed at the thought that Deirdre...,3
3,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Why did Blake create the three female super-im...,[He feels guilty about having slept with Eldor...,4
4,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Sabrina York is,"[a criminal that Blake is hunting, a psycheye ...",1
5,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Why doesn't Blake haggle with Eldoria about th...,"[He's afraid that if he angers her, she'll rev...",4


In [108]:
merged_df = pd.merge(question_df, experiment_df1, left_index=True, right_index=True)

# Keep only specific columns
desired_columns = ['article', 'question', 'options', 'predictions', 'labels', 'selected_chunk_ids']

merged_df  = merged_df[desired_columns]
merged_df.loc[:5]

,article,question,options,predictions,labels,selected_chunk_ids
0,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,How much time has passed between Blake's night...,"[7 years, 10 hours, 12 years, 1 hour]",2,1,"[5, 22, 33, 34, 59, 64, 86, 96, 98, 113, 157, ..."
1,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Why does Deirdre get so upset when Blake Past ...,[Because Blake is trying to guilt Deirdre into...,2,1,"[195, 212, 217, 219, 245, 284, 289, 292, 300, ..."
2,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Why does shame flame in Blake's cheeks when De...,[He is embarrassed at the thought that Deirdre...,0,2,"[375, 380, 392, 403, 404, 412, 414, 442, 469, ..."
3,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Why did Blake create the three female super-im...,[He feels guilty about having slept with Eldor...,3,3,"[565, 607, 678, 680, 682, 688, 709, 712, 717, ..."
4,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Sabrina York is,"[a criminal that Blake is hunting, a psycheye ...",0,0,"[799, 809, 822, 824, 826, 834, 836, 838, 848, ..."
5,THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...,Why doesn't Blake haggle with Eldoria about th...,"[He's afraid that if he angers her, she'll rev...",3,3,"[929, 935, 938, 947, 951, 958, 959, 969, 985, ..."


# 1. Finding out which type of questions are harder to predict (How/What/Who questions?)

In [98]:
def filterByQuestionType(df, question_type):
    return df[df['question'].str.lower().str.split().str[0] == question_type]

In [106]:
question_types = question_df['question'].str.lower().str.split().str[0].unique()
#question_types = ['why']

question_tuples = []
print(f'total: { question_df.shape[0]}')
for question_type in question_types:
    filtered_df = filterByQuestionType(merged_df, question_type)
    total_cnt = filtered_df.shape[0]
    
    correct_cnt = len(filtered_df[filtered_df['labels'] == filtered_df['predictions']])
    
    row = (question_type, total_cnt, correct_cnt)
    question_tuples.append(row)
    #print(row)
    

question_tuples = sorted(question_tuples, key=lambda x: x[1], reverse=True) # sort by total count

# only keep question types with tuples count larger than 10
thres = 5
question_tuples = list(filter(lambda x: x[1] > thres, question_tuples))
for (question_type, total_cnt, correct_cnt) in question_tuples:
    print(f'{question_type}: total={total_cnt}, correct={correct_cnt/total_cnt:.3f}')

total: 2086
what: total=809, correct=0.549
why: total=516, correct=0.552
how: total=332, correct=0.494
which: total=93, correct=0.484
who: total=58, correct=0.276
the: total=35, correct=0.600
of: total=34, correct=0.647
where: total=33, correct=0.455
according: total=22, correct=0.636
when: total=17, correct=0.471
in: total=13, correct=0.308
does: total=12, correct=0.667
if: total=9, correct=0.333
is: total=8, correct=0.625
did: total=6, correct=0.500
